In [19]:
import cv2
import os
import random
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
Adam = tf.keras.optimizers.Adam
to_categorical = tf.keras.utils.to_categorical
ImageDataGenerator = tf.keras.preprocessing.image.ImageDataGenerator
fashion_mnist = tf.keras.datasets.fashion_mnist
load_img = tf.keras.preprocessing.image.load_img
img_to_array = tf.keras.preprocessing.image.img_to_array

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import plotly.graph_objs as go
from plotly import subplots
import plotly

from sklearn.metrics import confusion_matrix
import plotly.figure_factory as ff
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings('ignore')

In [29]:
IMG_ROWS = 80
IMG_COLS = 80
NUM_CLASSES = 2
VAL_SIZE = 0.2
RANDOM_STATE = 99
BATCH_SIZE = 256

In [40]:
DIRECTORY_TRAIN = r'C:\dataprocess\clean and messy\train'
DIRECTORY_TEST = r'C:\dataprocess\clean and messy\test'
CATEGORIES = ['clean','messy']

In [41]:
for category in CATEGORIES:
    folder = os.path.join(DIRECTORY_TRAIN, category)
    print(folder)

C:\dataprocess\clean and messy\train\clean
C:\dataprocess\clean and messy\train\messy


In [31]:
def import_data(DIRECTORY):
    list_data = []
    for category in CATEGORIES:
        folder = os.path.join(DIRECTORY, category)
        label = CATEGORIES.index(category)
        for img in os.listdir(folder):
            img_path = os.path.join(folder, img)
            img_arr = cv2.imread(img_path)
            img_arr = cv2.resize(img_arr, (IMG_ROWS, IMG_COLS))
            img_arr = cv2.cvtColor(img_arr, cv2.COLOR_BGR2GRAY)
            list_data.append([img_arr, label])
    return list_data

In [32]:
Train_data = import_data(DIRECTORY_TRAIN)
Test_data = import_data(DIRECTORY_TEST)

error: OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-q3d_8t8e\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [27]:
print(len(Train_data))
print(len(Test_data))

NameError: name 'Train_data' is not defined

In [28]:
random.shuffle(Train_data)
random.shuffle(Test_data)

NameError: name 'Train_data' is not defined

In [ ]:
def image_vactor(data):
  feat = []
  y = []
  for features, labels in data:
    feat.append(features)
    y.append(labels)
  feat = np.array(feat)
  y = np.array(y)
  return feat, y

In [ ]:
train_data, y = image_vactor(Train_data)
test_data, y_test = image_vactor(Test_data)

In [ ]:
train_data.shape

In [ ]:
print("clean and messy train -  rows:",train_data.shape[0]," columns:", train_data.shape[1], " rows:", train_data.shape[2])
print("clean and messy test -  rows:",test_data.shape[0]," columns:", test_data.shape[1], " rows:", train_data.shape[2])

In [ ]:
for i in range(9):
    plt.subplot(330 + 1 + i)    
    plt.imshow(train_data[i],cmap='gray', vmin=0, vmax=255)

plt.tight_layout()
plt.savefig('clean and messy.jpeg', dpi=300)

In [ ]:
print(train_data.shape, test_data.shape, y.shape, y_test.shape)

In [ ]:
unique, counts = np.unique(y, return_counts=True)
dict(zip(unique, counts))

In [ ]:
unique, counts = np.unique(y_test, return_counts=True)
dict(zip(unique, counts))

In [ ]:
print(train_data.shape, test_data.shape)

train_data = train_data.reshape((train_data.shape[0], 80, 80, 1))
test_data = test_data.reshape((test_data.shape[0], 80, 80, 1))

print(train_data.shape, test_data.shape)

In [ ]:
train_data = train_data / 255.0
test_data = test_data / 255.0

In [ ]:
print(y.shape, y_test.shape)
print(y[:10])

In [ ]:
y = to_categorical(y)
y_test = to_categorical(y_test)

print(y.shape, y_test.shape)
y[:10]

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(train_data, y, test_size=VAL_SIZE, random_state=RANDOM_STATE)

x_train.shape, x_val.shape, y_train.shape, y_val.shape

In [ ]:
unique, counts = np.unique(y_val, return_counts=True)
dict(zip(unique, counts))

#Baseline Model

In [ ]:
# model = Sequential()
model = tf.keras.Sequential()

#1. CNN LAYER
model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), padding = 'Same', input_shape=(80, 80, 1)))
model.add(tf.keras.layers.Activation("relu"))

#2. CNN LAYER
model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), padding = 'Same'))
model.add(tf.keras.layers.Activation("relu"))

model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

#3. CNN LAYER
model.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same'))
model.add(tf.keras.layers.Activation("relu"))

#4. CNN LAYER
model.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same'))
model.add(tf.keras.layers.Activation("relu"))

model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

#FULLY CONNECTED LAYER
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.Activation("relu"))

#OUTPUT LAYER
model.add(tf.keras.layers.Dense(2, activation='softmax'))

In [ ]:
optimizer = Adam()
model.compile(optimizer = optimizer, loss = "BinaryCrossentropy", metrics=["accuracy"])

model.summary()

In [ ]:
NO_EPOCHS = 300

history = model.fit(x_train, y_train,
                  batch_size=BATCH_SIZE,
                  epochs=NO_EPOCHS,
                  verbose=1,
                  validation_data=(x_val, y_val))

In [ ]:
def create_trace(x,y,ylabel,color):
        trace = go.Scatter(
            x = x,y = y,
            name=ylabel,
            marker=dict(color=color),
            mode = "markers+lines",
            text=x
        )
        return trace
    
def plot_accuracy_and_loss(train_model):
    hist = train_model.history
    acc = hist['accuracy']
    val_acc = hist['val_accuracy']
    loss = hist['loss']
    val_loss = hist['val_loss']
    epochs = list(range(1,len(acc)+1))
    
    trace_ta = create_trace(epochs,acc,"Training accuracy", "Green")
    trace_va = create_trace(epochs,val_acc,"Validation accuracy", "Red")
    trace_tl = create_trace(epochs,loss,"Training loss", "Blue")
    trace_vl = create_trace(epochs,val_loss,"Validation loss", "Magenta")
   
    fig = subplots.make_subplots(rows=1,cols=2, subplot_titles=('Training and validation accuracy',
                                                             'Training and validation loss'))
    fig.append_trace(trace_ta,1,1)
    fig.append_trace(trace_va,1,1)
    fig.append_trace(trace_tl,1,2)
    fig.append_trace(trace_vl,1,2)
    fig['layout']['xaxis'].update(title = 'Epoch')
    fig['layout']['xaxis2'].update(title = 'Epoch')
    fig['layout']['yaxis'].update(title = 'Accuracy', range=[0,1])
    fig['layout']['yaxis2'].update(title = 'Loss', range=[0,1])

    plotly.offline.iplot(fig, filename='accuracy-loss')

In [ ]:
plot_accuracy_and_loss(history)

In [ ]:
score = model.evaluate(test_data, y_test,verbose=0)
print("Test Loss:",score[0])
print("Test Accuracy:",score[1])

In [ ]:
predicted_classes = model.predict(test_data)
predicted_classes = np.argmax(predicted_classes, axis=1)

In [ ]:
y_true = np.argmax(y_test, axis=1)

In [ ]:
cm = confusion_matrix(y_true, predicted_classes)
cm

In [ ]:
def cm_plot(cm, labels):
    x = labels
    y = labels

    z_text = [[str(y) for y in x] for x in cm]
    fig = ff.create_annotated_heatmap(cm, x=x, y=y, annotation_text=z_text, colorscale='blues')

    fig.update_layout(title_text='Confusion Matrix')

    fig.add_annotation(dict(font=dict(color="black",size=13),
                            x=0.5,
                            y=-0.15,
                            showarrow=False,
                            text="Predicted Value",
                            xref="paper",
                            yref="paper"
                           ))

    fig.add_annotation(dict(font=dict(color="black",size=13),
                            x=-0.20,
                            y=0.5,
                            showarrow=False,
                            text="Real Value",
                            textangle=-90,
                            xref="paper",
                            yref="paper"
                           ))

    fig.update_layout(margin=dict(t=50, l=200))
    fig['layout']['yaxis']['autorange'] = "reversed"

    fig['data'][0]['showscale'] = True
    fig.show()

In [ ]:
cm_plot(cm,CATEGORIES)

In [ ]:
print(classification_report(y_true, predicted_classes, target_names=CATEGORIES, digits=4))

#Image Augmentation

In [ ]:
datagen = ImageDataGenerator(
        rotation_range=0.05,    #Randomly rotate images in the range
        zoom_range = 0.2,       #Randomly zoom image                
        width_shift_range=0.1,  #Randomly shift images horizontally
        height_shift_range=0.1, #Randomly shift images vertically
        shear_range=0.05        #Randomly shear images
)

datagen.fit(x_train)

In [ ]:
x_batch = datagen.flow(x_train, y_train, batch_size=9).next()
x_batch[0].shape

In [ ]:
x_batch = x_batch[0].reshape((x_batch[0].shape[0], 80, 80))
x_batch.shape

In [ ]:
for i in range(0, 9):
    plt.subplot(330 + 1 + i)
    plt.imshow(x_batch[i], cmap=plt.get_cmap('gray'))
    
plt.savefig('waste2.jpeg', dpi=300)

In [ ]:
# model = Sequential()
model = tf.keras.Sequential()

#1. CNN LAYER
model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), padding = 'Same', input_shape=(80, 80, 1)))
model.add(tf.keras.layers.Activation("relu"))

#2. CNN LAYER
model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), padding = 'Same'))
model.add(tf.keras.layers.Activation("relu"))

model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

#3. CNN LAYER
model.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same'))
model.add(tf.keras.layers.Activation("relu"))

#4. CNN LAYER
model.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same'))
model.add(tf.keras.layers.Activation("relu"))

model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

#FULLY CONNECTED LAYER
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.Activation("relu"))

#OUTPUT LAYER
model.add(tf.keras.layers.Dense(2, activation='softmax'))

In [ ]:
optimizer = Adam()
model.compile(optimizer = optimizer, loss = "BinaryCrossentropy", metrics=["accuracy"])

model.summary()

In [ ]:
NO_EPOCHS = 300

history = model.fit(datagen.flow(x_train, y_train, batch_size=BATCH_SIZE),
                              shuffle=True,
                              epochs=NO_EPOCHS, validation_data = (x_val, y_val),
                              verbose=1, steps_per_epoch=x_train.shape[0] // BATCH_SIZE)

In [ ]:
plot_accuracy_and_loss(history)

In [ ]:
score = model.evaluate(test_data, y_test,verbose=0)
print("Test Loss:",score[0])
print("Test Accuracy:",score[1])

In [ ]:
predicted_classes = model.predict(test_data)
predicted_classes = np.argmax(predicted_classes, axis=1)

In [ ]:
y_true = np.argmax(y_test, axis=1)

In [ ]:
cm = confusion_matrix(y_true, predicted_classes)
cm

In [ ]:
cm_plot(cm,CATEGORIES)

In [ ]:
print(classification_report(y_true, predicted_classes, target_names=CATEGORIES, digits=4))

#Batch Normalization

In [ ]:
datagen = ImageDataGenerator(
        rotation_range=0.05,    #Randomly rotate images in the range
        zoom_range = 0.2,       #Randomly zoom image
        width_shift_range=0.1,  #Randomly shift images horizontally
        height_shift_range=0.1, #Randomly shift images vertically
        shear_range=0.05
)

datagen.fit(x_train)

In [ ]:
# model = Sequential()
model = tf.keras.Sequential()

#1. CNN LAYER
model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), padding = 'Same', input_shape=(80, 80, 1)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation("relu"))

#2. CNN LAYER
model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), padding = 'Same'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation("relu"))

model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

#3. CNN LAYER
model.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation("relu"))

#4. CNN LAYER
model.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation("relu"))

model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

#FULLY CONNECTED LAYER
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation("relu"))

#OUTPUT LAYER
model.add(tf.keras.layers.Dense(2, activation='softmax'))

In [ ]:
optimizer = Adam()
model.compile(optimizer = optimizer, loss = "BinaryCrossentropy", metrics=["accuracy"])

model.summary()

In [ ]:
NO_EPOCHS = 300
history = model.fit(datagen.flow(x_train, y_train, batch_size=BATCH_SIZE),
                              shuffle=True,
                              epochs=NO_EPOCHS, validation_data = (x_val, y_val),
                              verbose=1, steps_per_epoch=x_train.shape[0] // BATCH_SIZE)

In [ ]:
plot_accuracy_and_loss(history)

In [ ]:
score = model.evaluate(test_data, y_test,verbose=0)
print("Test Loss:",score[0])
print("Test Accuracy:",score[1])

In [ ]:
predicted_classes = model.predict(test_data)
predicted_classes = np.argmax(predicted_classes, axis=1)

In [ ]:
y_true = np.argmax(y_test, axis=1)

In [ ]:
cm = confusion_matrix(y_true, predicted_classes)
cm

In [ ]:
cm_plot(cm,CATEGORIES)

In [ ]:
print(classification_report(y_true, predicted_classes, target_names=CATEGORIES, digits=4))

#Dropout

In [ ]:
datagen = ImageDataGenerator(
        rotation_range=0.05,    #Randomly rotate images in the range
        zoom_range=0.2,         #Randomly zoom image
        width_shift_range=0.1,  #Randomly shift images horizontally
        height_shift_range=0.1, #Randomly shift images vertically
        shear_range=0.05        #Randomly shear images
)

datagen.fit(x_train)

In [ ]:
# model = Sequential()
model = tf.keras.Sequential()

#1. CNN LAYER
model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), padding = 'Same', input_shape=(80, 80, 1)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.Dropout(0.2))

#2. CNN LAYER
model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), padding = 'Same'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation("relu"))

model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.2))

#3. CNN LAYER
model.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.Dropout(0.2))

#4. CNN LAYER
model.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation("relu"))

model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.2))


#FULLY CONNECTED LAYER
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.Dropout(0.20))

#OUTPUT LAYER
model.add(tf.keras.layers.Dense(2, activation='softmax'))

In [ ]:
optimizer = Adam()
model.compile(optimizer = optimizer, loss = "BinaryCrossentropy", metrics=["accuracy"])

model.summary()

In [ ]:
NO_EPOCHS = 300

history = model.fit(datagen.flow(x_train, y_train, batch_size=BATCH_SIZE),
                              shuffle=True,
                              epochs=NO_EPOCHS, validation_data = (x_val, y_val),
                              verbose = 1, steps_per_epoch=x_train.shape[0] // BATCH_SIZE)

In [ ]:
plot_accuracy_and_loss(history)

In [ ]:
score = model.evaluate(test_data, y_test,verbose=0)
print("Test Loss:",score[0])
print("Test Accuracy:",score[1])

In [ ]:
predicted_classes = model.predict(test_data)
predicted_classes = np.argmax(predicted_classes, axis=1)

In [ ]:
y_true = np.argmax(y_test, axis=1)

In [ ]:
cm = confusion_matrix(y_true, predicted_classes)
cm

In [ ]:
cm_plot(cm,CATEGORIES)

In [ ]:
print(classification_report(y_true, predicted_classes, target_names=CATEGORIES, digits=4))